# Preliminary: SHAP and XGBoost 

In [ ]:

import shap
import xgboost
from sklearn.model_selection import KFold, train_test_split, KFold
from sklearn.metrics import mean_squared_error, make_scorer
import pandas as pd
import numpy as np
# import cupy as cp
from sklearn.model_selection import GridSearchCV


## Load dataset


In [ ]:
waterqualitypath = "../data/plankton-patrol/Plankton Patrol/Data/plankton-patrol_ChesapeakeWaterQuality.csv"
data = pd.read_csv(waterqualitypath)
data.info()

### Inspect column names

In [ ]:
data.columns

In [ ]:
data["Qualifier"] = data["Qualifier"].replace(np.nan, "=")


columns_to_exclude = ["Parameter", "MeasureValue", "Unit"]
unique_columns = [col for col in data.columns if col not in columns_to_exclude]

df_unique = data[unique_columns].drop_duplicates(subset="EventId")
print(df_unique.shape, data.shape)
data_r = data.pivot_table(
    index=["EventId"], columns="Parameter", values="MeasureValue", aggfunc="first"
).reset_index()
exclude_from_pivoted = ["Parameter", "MeasureValue", "Unit", "SampleDate", "SampleTime"]
pivoted_columns = data_r.columns.tolist()
for ce in exclude_from_pivoted:
    if ce in pivoted_columns:
        pivoted_columns.remove(ce)
data_m = pd.merge(df_unique, data_r, on="EventId", how="left")


print(data_m.columns, data_m.shape)

In [ ]:
columns_to_drop = ['EventId']

# Drop the specified columns
data_md = data_m.drop(columns=columns_to_drop)


In [ ]:
data_md.head()

In [ ]:
pivoted_columns

In [ ]:
set_numeric = [
    "TotalDepth",
    "UpperPycnocline",
    "LowerPycnocline",
    "Depth",
    "Latitude",
    "Longitude",
    "MeasureValue",
    "CHLA",
    "DIN",
    "DO",
    "DOC",
    "DON",
    "DOP",
    "DO_SAT_P",
    "FSS",
    "KD",
    "NH4F",
    "NO23F",
    "NO2F",
    "NO3F",
    "PC",
    "PH",
    "PHEO",
    "PIP",
    "PN",
    "PO4F",
    "PP",
    "Parameter",
    "SALINITY",
    "SECCHI",
    "SIF",
    "SIGMA_T",
    "SPCOND",
    "TDN",
    "TDP",
    "TN",
    "TON",
    "TP",
    "TSS",
    "TURB_NTU",
    "VSS",
    "WTEMP",
]
# set_string = ['
set_date = ["SampleDate", "SampleTime"]


def combine_date_time_strings(df, date_col, time_col):
    # Combine date and time strings
    combined_col = df[date_col] + " " + df[time_col]
    # Convert the combined string to datetime
    datetime_col = "Sample"

    df[datetime_col] = pd.to_datetime(combined_col, errors="coerce")
    df[datetime_col + "_year"] = df[datetime_col].dt.year
    df[datetime_col + "_month"] = df[datetime_col].dt.month
    df[datetime_col + "_day"] = df[datetime_col].dt.day
    df[datetime_col + "_hour"] = df[datetime_col].dt.hour
    df[datetime_col + "_minute"] = df[datetime_col].dt.minute
    df[datetime_col + "_second"] = df[datetime_col].dt.second
    newcolnames = [
        f"{datetime_col}_year",
        f"{datetime_col}_month",
        f"{datetime_col}_day",
        f"{datetime_col}_hour",
        f"{datetime_col}_minute",
        f"{datetime_col}_second",
    ]
    # Drop the original date, time, and combined datetime columns
    df.drop(columns=[date_col, time_col, datetime_col], inplace=True)

    return df, newcolnames


# Function to convert columns to appropriate types
def convert_dtypes(df):
    for col in df.columns:
        df[col] = df[col].replace("nan", np.nan)
        # print(df[col][564463])
        print(f"converting column {col}", end="\t")
        print(df[col].dtype)

        if col not in (set_numeric + set_date):
            df[col] = pd.Categorical(df[col])
            print("Categorical")
        elif col in set_numeric:
            # try:
            # Try converting to numeric (float)
            df[col] = pd.to_numeric(df[col], errors="coerce")
            print("Numeric")
        # except (ValueError, TypeError):
        # try:
        elif col in set_date:
            print("Date, skipped")
            pass
            # Try converting to datetime (date)
            # df[col] = pd.to_datetime(df[col], errors='coerce')
            # except (ValueError, TypeError):
            # Check for categorical
            # unique_ratio = df[col].nunique() / df[col].count()
            # if unique_ratio < 0.2:  # heuristic for categorical, adjustable threshold
            # Convert to string if not numeric, date, or categorical
        else:
            print(f"{col}: string")
            df[col] = df[col].astype(str)
            print("string")
        print(df[col].dtype)
    return df


# Apply the conversion function
datacopy = data_md.__deepcopy__()
data_conv = convert_dtypes(datacopy)
data_cleana, newcols = combine_date_time_strings(data_conv, "SampleDate", "SampleTime")
# Check the result
print(data_cleana.dtypes)
print(data_cleana.shape)
# data.dropna(axis='TotalDepth',how='all')
data_cleana = data_cleana[:-1]
print(data_cleana.shape)

# data_conv = convert_dtypes(data_conv)
# for col in data.columns:
#     # print(data[col])
#     if col not in (set_float + set_date):
#         data[col]= pd.Categorical(df[col])(data[col][1:])
#     # if col in set_string:
#     #     data[col][1:]=data[col][1:].astype("string")
#     if col in set_float:
#         data[col]= pd.to_numeric(data[col], errors='raise')
#     if col in set_date:
#         print(col)
#         data[col][1:]=pd.to_datetime(data[col][1:],errors='coerce')
#         # data[col]=data[col].dt.strftime(f'%m/%d/%Y')
data_cleana

In [ ]:
pivoted_columns= pivoted_columns+newcols
print(len(pivoted_columns), pivoted_columns)

In [ ]:
# pivoted_columns = [col for col in pivoted_columns if col in data_clean.columns]

# data_clean = data_clean.replace([np.inf, -np.inf], np.nan).dropna(
#     subset=[pivoted_columns]
# )
data_cleana = data_cleana.replace([np.inf, -np.inf], np.nan)


missing_percentage = data_cleana.isnull().mean()
print(missing_percentage)
clean_columns = data_cleana.columns.tolist()

# Drop columns where more than 95% of the data is missing
columns_to_drop = missing_percentage[missing_percentage > 0.90].index
print(columns_to_drop)
data_cleana = data_cleana.drop(columns=columns_to_drop)
data_clean = data_cleana.copy()
data_clean = data_cleana.dropna(subset=['CHLA'])
# data_clean = data_cleana.dropna(
#     subset=[
#         "CHLA",
#         "DIN",
#         "DO",
#         "DOC",
#         "DON",
#         "DOP",
#         "DO_SAT_P",
#         "FSS",
#         "KD",
#         "NH4F",
#         "NO23F",
#         "NO2F",
#         "NO3F",
#         "PC",
#         "PH",
#         "PHEO",
#         "PN",
#         "PO4F",
#         "PP",
#         "SALINITY",
#         "SECCHI",
#         "SIF",
#         "SIGMA_T",
#         "SPCOND",
#         "TDN",
#         "TDP",
#         "TN",
#         "TON",
#         "TP",
#         "TSS",
#         "VSS",
#         "WTEMP",
#     ]
# )
dropped_columns = [col for col in clean_columns if col not in data_clean.columns]

data_clean.info()

In [ ]:
print(dropped_columns)

In [ ]:

X, y = data_clean.drop(['CHLA'], axis=1), data_clean['CHLA']
# X = pd.get_dummies(X, drop_first=True) # TOO much memory
# Label encode categorical variables
for col in X.select_dtypes(include=['category']).columns:
    X[col] = X[col].cat.codes
X.shape, y.shape


# Visualize data

In [ ]:
import matplotlib.pyplot as plt
def visualize_param_map(df, parameter, path):
    latitude = df['Latitude']
    longitude = df['Longitude']
    cha = df[parameter]

    # Create a scatter plot
    plt.figure(figsize=(10, 6))
    scatter = plt.scatter(longitude, latitude, c=cha, cmap='coolwarm', marker='o')

    # Add a color bar
    plt.colorbar(scatter, label='CHA Values')

    # Add labels and title
    plt.title(f'{parameter} Map')
    plt.xlabel('Longitude')
    plt.ylabel('Latitude')

    # Display the plot
    plt.grid(True)
    # plt.show()
    plt.savefig(path)

Visualize parameters

In [ ]:
listparams =['DIN', 'DO', 'DOC', 'DON', 'DOP',
       'DO_SAT_P', 'FSS', 'KD', 'NH4F', 'NO23F', 'NO2F', 'NO3F', 'PC', 'PH',
       'PHEO', 'PN', 'PO4F', 'PP', 'SALINITY', 'SECCHI',
       'SIF', 'SIGMA_T', 'SPCOND', 'TDN', 'TDP', 'TN', 'TON', 'TP', 'TSS',
       'VSS', 'WTEMP']


# import os
# path = "../data/visualizations"
# for p in listparams:
#     visualize_param_map(data_clean, p, os.path.join(path, f"{p}.png"))

In [ ]:
# df = X_train
# print("Original DataFrame:")
# print(df)

# # Identify NaNs in the DataFrame
# nan_locations = df.isna()
# print("\nLocations of NaNs in the DataFrame:")
# print(nan_locations)

# # Count NaNs in each column
# nan_count_per_column = df.isna().sum()
# print("\nCount of NaNs in each column:")
# print(nan_count_per_column)

# # Count NaNs in each row
# nan_count_per_row = df.isna().sum(axis=1)
# print("\nCount of NaNs in each row:")
# print(nan_count_per_row)

# # Rows with at least one NaN
# rows_with_nans = df[df.isna().any(axis=1)]
# print("\nRows with at least one NaN:")
# print(rows_with_nans)

# # Columns with at least one NaN
# columns_with_nans = df.columns[df.isna().any()].tolist()
# print("\nColumns with at least one NaN:")
# print(columns_with_nans)

# 3. Train an XGBoost model

In [ ]:
# Define the parameter grid to search over
# model = xgboost.XGBRegressor(
#     max_depth=3,  # Maximum depth of a tree
#     learning_rate=0.1,  # Learning rate
#     n_estimators=100,  # Number of trees
#     subsample=0.8,  # Subsample ratio of the training instances
#     colsample_bytree=0.8,  # Subsample ratio of columns when constructing each tree
#     gamma=0,  # Minimum loss reduction required to make a further partition
#     min_child_weight=1,  # Minimum sum of instance weight needed in a child
#     device="cuda",
#     enable_categorical=True,
# )
param_grid = {
    "max_depth": [3, 5, 7],
    "learning_rate": [0.1, 0.01, 0.001, 0.0001],
    "n_estimators": [50, 100, 200],
}
models = {
    "LightGBM": (
        lgb.LGBMRegressor(),
        {
            "max_depth": [3, 5, 7],
            "n_estimators": [50, 100, 200],
            "learning_rate": [0.1, 0.01, 0.001, 0.0001]
        },
    ),
    "XGBoost": (
        xgb.XGBRegressor(),
        {
            "max_depth": [3, 5, 7],
            "n_estimators": [50, 100, 200],
            "learning_rate": [0.1, 0.01, 0.001, 0.0001]
        },
    ),
    "MLPRegressor": (
        MLPRegressor(max_iter=1000),
        {"hidden_layer_sizes": [(50, 50), (100,)], "learning_rate_init": [0.001, 0.01]},
    ),
}

# models = {
#     'LightGBM': lgb.LGBMRegressor(),
#     'XGBoost': xgb.XGBRegressor(),
#     'MLPRegressor': CatBoostRegressor(silent=True)
# }
# Make custom scorer
mse_scorer = make_scorer(mean_squared_error, greater_is_better=False)
mse_scores = []
kfold_bestparams = []
allshapvalues = []
best_models = {}
kf = KFold(n_splits=5)
for model_name, (model, param_grid) in models.items():


    # Create a GridSearchCV object
    grid_search = GridSearchCV(
        estimator=model, param_grid=param_grid, cv=kf, scoring=mse_scorer
    )

    # X_train, X_test, y_train, y_test = train_test_split(
    #     X, y, test_size=0.2, random_state=42
    # )
    # Fit the grid search to the training data
    # grid_search.fit(X_train, y_train)
    grid_search.fit(X, y)

    # Get the best parameters
    # best_params = grid_search.best_params_
    # print(f"Best Parameters: {best_params}")

    # Create a new model with the best parameters
    # best_model = model(**best_params)
    # # Train the best model
    # best_model.fit(X_train, y_train)
    best_model = grid_search.best_estimator_

    # Store the best model and its parameters
    best_models[model_name] = {
        "best_estimator": best_model,
        "best_params": grid_search.best_params_,
        "best_score": grid_search.best_score_
    }
    # Predict on the test set
    # y_pred_best = best_model.predict(X_test)
    # mse = mean_squared_error(y_test, y_pred_best)

    # Evaluate the best model
    # accuracy_best = (y_pred_best == y_test).mean()
    
    print(f"MSE with Best Parameters: {best_models[model_name]['best_score']}")
    mse_scores.append(best_models[model_name]['best_score'])
    kfold_bestparams.append(best_models[model_name]['best_params'])

    # plt.figure()
    explainer = shap.Explainer(model=best_model, masker=X)
    # plt.savefig(
    # f"shap_explainer_best_{model_name}_fold-{i}.png", bbox_inches="tight"
    # )
    # plt.close()
    # for i, (train_index, test_index) in enumerate(kf.split(X)):
    #     X_train, X_test, y_train, y_test = (
    #         X.iloc[train_index],
    #         X.iloc[test_index],
    #         y.iloc[train_index],
    #         y.iloc[test_index],
    #     )
    shap_values = explainer(X)
    allshapvalues.append(shap_values)
    explainer.__class__

    # plt.figure()
    shap.summary_plot(shap_values, X, show=False)
    # plt.savefig(
    #     f"shap_summary_plot_best_{model_name}_fold-{i}.png", bbox_inches="tight"
    # )
    # plt.close()

In [ ]:
for shap_values in allshapvalues:
    # shap.plots.waterfall(shap_values[50], max_display=58)
    # shap.initjs()
    shap.plots.bar(shap_values, max_display=11)

    # shap.plots.beeswarm(shap_values, max_display=58)
    shap.plots.beeswarm(shap_values.abs, max_display=11)

In [ ]:

# most_freq = max(set(kfold_bestparams.items()), key=kfold_bestparams.count)

print(44.31 + 79.62)
print(38.40+89.59)
# for bestparams,msescore in zip(kfold_bestparams, mse_scores):
        
# # params = dict()
# # params["device"] = "cuda"
# # params["tree_method"] = "hist"

# model = xgboost.XGBRegressor(missing=np.nan, enable_categorical=True, device="cuda")
# model.fit(X_train, y_train)
# grid_search.cv_results_.keys()

## 4.1. Estimate the Shapley values

In [ ]:


explainer = shap.Explainer(model=model, masker=X_train)
explainer.__class__


In [ ]:
shap.initjs()

In [ ]:
shap.plots.force(shap_values[0])

In [ ]:
shap.plots.bar(shap_values, max_display=10)

In [ ]:

shap.initjs()
shap.plots.force(shap_values)


In [ ]:
shap.plots.beeswarm(shap_values,max_display=58)

In [ ]:
shap.plots.scatter(shap[:, "PN"], color=shap_values)